# Using a useful subpackage of regional_mom6! mom6_bathy!



For this example we need:

- A working experiment object! Try running one of the previous demos!
- [GEBCO bathymetry](https://www.gebco.net/data_and_products/gridded_bathymetry_data/)



## What does the `mom6_bathy` package do?

mom6_bathy underwrites the grid-generation parts of regional_mom6. Previously, regional_mom6 had grid generation inside of itself. It's now part of mom6_bathy to take advantage of a few cool features inside of that package, some of which we'll show below! If you're interested in more features than shown here (things like slicing grids, filling bathymetry with a spoon-type depth, writing CICE grids), look at mom6_bathy docs! We're up to date with them!

## What does this notebook do?
This notebook is designed to showcase some of the features of mom6_bathy that we get as part of regional_mom6. As such, we're only going to be needed grid information!

Input Type | Source | Subsets required
---|---|---
Bathymetry | [GEBCO](https://www.gebco.net/data_and_products/gridded_bathymetry_data/) | whole globe or subset around domain

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import regional_mom6 as rmom6
from pathlib import Path
import os


## Step 1: Choose our domain, define workspace paths

To make sure that things are working I'd recommend starting with the default example defined below. If this runs ok, then change to a domain of your choice and hopefully it runs ok too! If not, check the [README](https://github.com/COSIMA/regional-mom6/blob/main/README.md) and [documentation](https://regional-mom6.readthedocs.io/) for troubleshooting tips.

You can log in and use [this GUI](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/download) to find the lat/lon of your domain and copy paste below.

In [2]:
expt_name = "tasmania-example-reanalysis"

latitude_extent = [-48, -38.95]
longitude_extent = [143, 150]

date_range = ["2003-01-01 00:00:00", "2003-01-05 00:00:00"]

## Place where all your input files go 
input_dir = Path(f"mom6_input_directories/{expt_name}/")

## Directory where you'll run the experiment from
run_dir = Path(f"mom6_run_directories/{expt_name}/")

## if directories don't exist, create them
for path in (run_dir, input_dir):
    os.makedirs(str(path), exist_ok=True)

## Step 2: Make experiment object
The `regional_mom6.experiment` contains the regional domain basics, and also generates the horizontal and vertical grids, `hgrid` and `vgrid` respectively, and sets up the directory structures. 

In [3]:
expt = rmom6.experiment(
    longitude_extent = longitude_extent,
    latitude_extent = latitude_extent,
    date_range = date_range,
    resolution = 0.05,
    number_vertical_layers = 75,
    layer_thickness_ratio = 60,
    depth = 4500,
    minimum_depth = 5,
    mom_run_dir = run_dir,
    mom_input_dir = input_dir,
)

## Step 4: Set up bathymetry

Similarly to ocean forcing, we point the experiment's `setup_bathymetry` method at the location of the file of choice and also provide the variable names. We don't need to preprocess the bathymetry since it is simply a two-dimensional field and is easier to deal with. Afterwards you can inspect `expt.bathymetry` to have a look at the regional domain.

After running this cell, your input directory will contain other bathymetry-related things like the ocean mosaic and mask table too. The mask table defaults to a 10x10 layout and can be modified later.

In [6]:
expt.setup_bathymetry(
    bathymetry_path='/glade/campaign/cgd/oce/projects/CROCODILE/workshops/2025/CrocoDash/data/gebco/GEBCO_2024.nc',  #PATH_TO_GEBCO_FILE/GEBCO_2022.nc
    longitude_coordinate_name='lon',
    latitude_coordinate_name='lat',
    vertical_coordinate_name='elevation',
    )

Setting up bathymetry...if this fails, please follow the printed instructions with your experiment topo object, like this: [experiment_obj].topo 
**NOTE**
            If bathymetry setup fails (e.g. kernel crashes), restart the kernel and edit this cell.
            Call ``[topo_object_name].mpi_set_from_dataset()`` instead. Follow the given instructions for using mpi 
            and ESMF_Regrid outside of a python environment. This breaks up the process, so be sure to call
            ``[topo_object_name].tidy_dataset() after regridding with mpi.
Begin regridding bathymetry...

Original bathymetry size: 37.08 Mb
Regridded size: 0.84 Mb

Tidy bathymetry: Reading in regridded bathymetry to fix up metadata...done. Filling in inland lakes and channels... 

<xarray.Dataset>
Dimensions:  (ny: 249, nx: 140)
Dimensions without coordinates: ny, nx
Data variables:
    y        (ny, nx) float64 -47.98 -47.98 -47.98 ... -38.97 -38.97 -38.97
    x        (ny, nx) float64 143.0 143.1 143.1 143.2 ... 149.9 149.9 150.0
    mask     (ny, nx) int32 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    depth    (ny, nx) float64 4.592e+03 4.545e+03 ... 4.198e+03 4.307e+03
Attributes:
    date_created:  2025-10-22T13:59:29.766689
    title:         MOM6 topography file
    min_depth:     5
    max_depth:     5043.271093411908

### Try out the topo editor!

Calling `expt.topo` returns the mom6_bathy topo object, which can be used in the topo editor. 

In [7]:
%matplotlib ipympl
from mom6_bathy.topo_editor import TopoEditor
TopoEditor(expt.topo)

TopoEditor(children=(VBox(children=(HTML(value='<h2>Topo Editor</h2>'), HTML(value='<hr><h3>Display</h3>'), To…

In [13]:
# After editing the topo, you can save the topo by writing it into your input directory
expt.topo.write_topo(expt.mom_input_dir / "bathymetry.nc") # This saves the file 

## Writing out alternate files!

ESMF file: If you run with the NOUPC Coupler, you need to generate an ESMF File, which contains all kinds of grid and mask information.

CICE file: If you run with the sea-ice model CICE, you need to generate a CICE specific file written on a Arakawa B grid. (until they update to use MOM6 style grids/bathy!)

In [ ]:
# Write out an ESMF File
expt.topo.write_esmf_mesh(expt.mom_input_dir / "ESMF_grid.nc")

# Write out a CICE grid File
expt.topo.write_cice_grid(expt.mom_input_dir / "CICE_grid.nc")